## 微珠散射介质模拟

### 微珠散射介质构建

In [20]:
import numpy as np
import matplotlib.pyplot as plt

In [21]:
def generate_balls(n, area, r_range, random_weight):
    balls = []
    for i in range(n):
        x = np.random.randint(area[0, 0], area[0, 1])
        y = np.random.randint(area[1, 0], area[1, 1])
        z = np.random.randint(area[2, 0], area[2, 1])
        r = np.random.rand() * random_weight * r_range[1] + (1 - np.random.rand() * random_weight) * r_range[0]
        attempts = 0
        while attempts < 100:
            overlap = False
            for ball in balls:
                distance = np.sqrt((x - ball[0])**2 + (y - ball[1])**2 + (z - ball[2])**2)
                if distance < (r + ball[3]):
                    overlap = True
                    break
            if not overlap:
                break
            x = np.random.randint(area[0, 0], area[0, 1])
            y = np.random.randint(area[1, 0], area[1, 1])
            z = np.random.randint(area[2, 0], area[2, 1])
            r = np.random.rand() * random_weight * r_range[1] + (1 - np.random.rand() * random_weight) * r_range[0]
            attempts += 1
        balls.append([x, y, z, r])
    return balls

In [22]:
def light_source(grid_shape, amplitude, polarization, phase, wavelength, center, span):
    light_source = {
        'grid_shape': grid_shape,#波源的网格数目
        'positions': [], #所有网格的位置，一个[nx, ny, 3]的数组
        'amplitudes': amplitude, #振幅,一个[nx, ny]的数组
        'polarizations': polarization,#偏振角度，一个[nx, ny]的数组
        'phases': phase,#相位，一个[nx, ny]的数组
        'wavelength': wavelength,#波长
        'center': center,#中心位置,一个[3]的数组
        'span': span#范围，一个[2]的数组，表示二维波源的纵横尺寸
    }

    positions = np.zeros(grid_shape)
    for nx in range(grid_shape[0]):
        for ny in range(grid_shape[1]):
                x = nx * (span[0] / grid_shape[0]) - span[0]/2 + center[0]
                y = ny * (span[1] / grid_shape[1]) - span[1]/2 + center[1]
                z =center[2]
                positions[nx, ny] = [x, y, z]
    light_source['positions'] = positions
    
    return light_source

def plane_wave(gride_shape, A, polarization, phase, wavelength, center, span):
    amplitude = np.ones(gride_shape) * A
    phase = np.ones(gride_shape) * phase
    polarization = np.ones(gride_shape) * polarization
    return light_source(gride_shape, amplitude, polarization, phase, wavelength, center, span)

def gaussian_beam(grid_shape, A, polarization, phase, center,wavelength , span, waist):
    amplitude = np.zeros(grid_shape)
    for nx in range(grid_shape[0]):
        for ny in range(grid_shape[1]):
            x = nx * (span[0] / grid_shape[0]) - span[0]/2 + center[0]
            y = ny * (span[1] / grid_shape[1]) - span[1]/2 + center[1]
            r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
            amplitude[nx, ny] = A * np.exp(-(r**2) / (2 * waist**2))
        
    phase = np.ones(grid_shape) * phase
    polarization = np.ones(grid_shape) * polarization
    return light_source(grid_shape, amplitude, polarization, phase,wavelength, center, span) 


In [23]:
def monitor(grid_shape, center, span):
    light_source = {
        'grid_shape': grid_shape,#波源的网格数目
        'positions': [], #所有网格的位置，一个[nx, ny, 3]的数组
        'amplitudes': amplitude, #振幅,一个[nx, ny]的数组
        'polarizations': polarization,#偏振角度，一个[nx, ny]的数组
        'phases': phase,#相位，一个[nx, ny]的数组
        'center': center,#中心位置,一个[3]的数组
        'span': span#范围，一个[2]的数组，表示二维波源的纵横尺寸
    }

    positions = np.zeros(grid_shape)
    
    for nx in range(grid_shape[0]):
        for ny in range(grid_shape[1]):
                x = nx * (span[0] / grid_shape[0]) - span[0]/2 + center[0]
                y = ny * (span[1] / grid_shape[1]) - span[1]/2 + center[1]
                z =center[2]
                positions[nx, ny] = [x, y, z]

    light_source['positions'] = positions
    
    return light_source

In [24]:
def light_field_add(amplitude,phase,polarization):
    n = len(amplitude)
    total_amplitude = np.zeros_like(amplitude[0])
    total_phase = np.zeros_like(phase[0])
    total_polarization = np.zeros_like(polarization[0])

    for i in range(n):
        total_amplitude_x += amplitude[i] * np.exp(1j * phase[i])
    for i in range(n):
        total_amplitude_y += amplitude[i] * np.exp(1j * phase[i]) * np.cos(polarization[i])
    
    total_phase = np.angle(total_amplitude)
    total_amplitude = np.abs(total_amplitude_x)
    total_polarization = np.arctan2(np.real(total_amplitude), np.real(total_amplitude_y))

    return total_amplitude, total_phase, total_polarization

In [27]:
def fresnel_diffraction(light_source, x, y, z):
    k = 2 * np.pi / light_source['wavelength']  # 波数
    amplitude = np.zeros((len(x), len(y)), dtype=complex)
    
    for i, xi in enumerate(x):
        for j, yj in enumerate(y):
            for nx in range(light_source['grid_shape'][0]):
                for ny in range(light_source['grid_shape'][1]):
                    pos = light_source['positions'][nx, ny]
                    r = np.sqrt((xi - pos[0])**2 + (yj - pos[1])**2 + (z - pos[2])**2)
                    amplitude[i, j] += (light_source['amplitudes'][nx, ny] * 
                                        np.exp(1j * (k * r + light_source['phases'][nx, ny])) / r)
    
    return amplitude

# 示例调用
x = np.linspace(-10, 10, 100)
y = np.linspace(-10, 10, 100)
z = 10
light_source1 = plane_wave((100, 100), 1, 0, 0, 442, (0,0,0), (20, 20))
light_field = fresnel_diffraction(light_source1, x, y, z)

# 绘制结果
plt.figure(figsize=(8, 6))
plt.imshow(np.abs(light_field), extent=(x.min(), x.max(), y.min(), y.max()), cmap='viridis')
plt.colorbar(label='Amplitude')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Fresnel Diffraction Pattern')
plt.show()

ValueError: setting an array element with a sequence.